# Семантическое ядро

В этом примере кода вы будете использовать [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel), AI-фреймворк, чтобы создать базового агента.

Цель этого примера — показать вам шаги, которые мы позже будем использовать в дополнительных примерах кода при реализации различных агентных шаблонов.


## Импорт необходимых пакетов Python


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Создание клиента

В этом примере мы будем использовать [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) для доступа к LLM.

`ai_model_id` определен как `gpt-4o-mini`. Попробуйте изменить модель на другую, доступную в маркетплейсе GitHub Models, чтобы увидеть различные результаты.

Чтобы использовать `Azure Inference SDK`, который применяется для `base_url` в GitHub Models, мы будем использовать коннектор `OpenAIChatCompletion` в Semantic Kernel. Также существуют [другие доступные коннекторы](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion), которые позволяют использовать Semantic Kernel с другими поставщиками моделей.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Создание агента

Ниже мы создаем агента под названием `TravelAgent`.

В этом примере мы используем очень простые инструкции. Вы можете изменить эти инструкции, чтобы увидеть, как агент реагирует по-разному.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Запуск агента

Теперь мы можем запустить агента, определив поток типа `ChatHistoryAgentThread`. Все необходимые системные сообщения передаются агенту через аргумент `messages` ключевого слова invoke_stream.

После этого мы создаем `user_inputs`, который будет содержать сообщение, отправляемое пользователем агенту. В данном случае мы задали это сообщение как `Plan me a sunny vacation`.

Не стесняйтесь изменить это сообщение, чтобы увидеть, как агент отреагирует по-другому.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, имейте в виду, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
